In [ ]:
import os
import sys

os.chdir('..')

parent_directory = os.getcwd()
sys.path.append(parent_directory)

In [ ]:
from src.rea_beta_backend_toolsets.services.logging.logger import setup_logging

setup_logging()

In [ ]:
from src.rea_beta_backend_toolsets.services.requests.request import RequestManager

request = RequestManager()

In [ ]:
response = request.get("https://example.com/", timeout=20, use_proxy=False, use_user_agent=False)